<a href="https://colab.research.google.com/github/chizuchizu/IOAI/blob/main/Task2/chizu_007_task2_pretrain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import userdata

read_access_token = userdata.get('hf_read')
write_access_token = userdata.get('hf_write')

### Dependencies

In [2]:
import importlib
import torch, transformers

if '2.3.0' not in torch.__version__:
  !pip install torch==2.3.0
if transformers.__version__!='4.41.2':
  !pip install transformers==4.41.2

if importlib.util.find_spec('datasets') is None:
  !pip install datasets==2.18.0s
  !pip install evaluate==0.4.2
  !pip install accelerate -U

if importlib.util.find_spec("wandb") is None:
  !pip install wandb -q

If you've just installed `accelerate`, execute `Runtime > Restart session and run all` in the Colab UI menu above.

In [3]:
import os
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.functional import F

import torchvision
from torchvision import datasets, transforms, models

from tqdm.auto import tqdm

from transformers import AutoModelForSequenceClassification, AutoTokenizer, DataCollatorWithPadding, get_scheduler, BertForMaskedLM, BertTokenizer
from transformers import DataCollatorForLanguageModeling
from datasets import load_dataset, Dataset, DatasetDict

import torch.cuda.amp as amp # or import torch.cuda.amp as amp for PyTorch's native amp


import evaluate
import wandb

In [4]:
# ====================================================
# CFG
# ====================================================

class CFG:
    num_workers=4
    project = "IOAI_Task2"
    name = "Task2_chizu_007_pretrain"

    # model
    base_model_name = "google-bert/bert-base-multilingual-uncased"
    mlm_probability = 0.15

    # training
    epochs = 2

    scheduler='CosineAnnealingLR' # ['ReduceLROnPlateau', 'CosineAnnealingLR', 'CosineAnnealingWarmRestarts']

    lr = 5e-05

    # dataset
    max_length = 256

    # T4: 32
    # L4: 64
    train_batch_size = 32

    seed=42
    train=True

In [5]:
wandb.login(key=userdata.get('wandb_token'))

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [6]:
# for wandb
cfg = dict(vars(CFG))
cfg = {k: v for k, v in cfg.items() if "__" not in k}

In [7]:
# classification_dataset = load_dataset('InternationalOlympiadAI/NLP_problem', token=read_access_token)
# tokenizer = AutoTokenizer.from_pretrained(CFG.base_model_name)
brahmi_to_devanagari = {
    '𑀓': 'क', '𑀔': 'ख', '𑀕': 'ग', '𑀖': 'घ', '𑀗': 'ङ', '𑀘': 'च', '𑀙': 'छ',
    '𑀚': 'ज', '𑀛': 'झ', '𑀜': 'ञ', '𑀝': 'ट', '𑀞': 'ठ', '𑀟': 'ड', '𑀠': 'ढ',
    '𑀡': 'ण', '𑀢': 'त', '𑀣': 'थ', '𑀤': 'द', '𑀥': 'ध', '𑀦': 'न', '𑀧': 'प',
    '𑀨': 'फ', '𑀩': 'ब', '𑀪': 'भ', '𑀫': 'म', '𑀬': 'य', '𑀭': 'र', '𑀮': 'ल',
    '𑀯': 'व', '𑀰': 'श', '𑀱': 'ष', '𑀲': 'स', '𑀳': 'ह', '𑁦':'०', '𑁣': 'नब्बे'
}

def transliterate_brahmi_to_latin(text):
    transliterated_text = ''
    for char in text:
        if char in brahmi_to_devanagari:
            transliterated_text += brahmi_to_devanagari[char]
        else:
            transliterated_text += char
    return transliterated_text

def to_device(batch, device):
    output = {}
    for k, v in batch.items():
        try:
            output[k] = v.to(device)
        except:
            output[k] = v
    return output

In [8]:
def main():
    wandb.init(
        name=CFG.name,
        project=CFG.project,
        config=cfg
    )

    raw_dataset = load_dataset('InternationalOlympiadAI/NLP_problem_raw', token=read_access_token)
    tokenizer = AutoTokenizer.from_pretrained(CFG.base_model_name)

    data_collator = DataCollatorForLanguageModeling(
        tokenizer=tokenizer,
        mlm=True,
        mlm_probability=CFG.mlm_probability
    )

    def transform(example_batch):
        example_batch["text"] = [transliterate_brahmi_to_latin(x) for x in example_batch["text"]]
        inputs = tokenizer([x for x in example_batch["text"]],  truncation=True, max_length=CFG.max_length, padding="max_length", return_tensors="pt")
        return inputs

    tokenized_data = raw_dataset.with_transform(transform)


    # dataset
    train_dataset = tokenized_data["train"]
    train_loader = DataLoader(
        train_dataset,
        batch_size=CFG.train_batch_size,
        num_workers=0,
        pin_memory=True,
        shuffle=True,
        drop_last=True,
        collate_fn=data_collator,
    )

    model = BertForMaskedLM.from_pretrained(
        CFG.base_model_name
    ).cuda()
    num_training_steps = CFG.epochs * len(train_loader)
    optimizer = optim.AdamW(model.parameters(), lr=CFG.lr)
    scheduler = get_scheduler(name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

    scaler = torch.cuda.amp.GradScaler()

    def train_one_epoch(model, scheduler, train_loader, optimizer):
        model.train()
        running_loss = 0.0
        progress_bar = tqdm(train_loader, dynamic_ncols=True)

        for step, batch in enumerate(progress_bar):
            batch = to_device(batch, device)

            # Convert data to fp16
            with amp.autocast():
                outputs = model(**batch)
                loss = outputs.loss

            # Scale loss for fp16 training
            scaler.scale(loss).backward()
            text = f"step {step}, loss: {loss:.5f}"
            progress_bar.set_description(text)

            wandb.log(
                {
                    "train_loss": loss,
                    "lr": optimizer.param_groups[0]["lr"],
                    "step": step,
                }
            )

            # Optimizer step with gradient scaling
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
            scheduler.step()


    # Train and evaluate the model
    model.to(device)
    for i in range(CFG.epochs):
        train_one_epoch(model, scheduler, train_loader, optimizer)
        print(f'Epoch {i+1}')
    return model

In [9]:
model = main()

wandb: Currently logged in as: asiatic-cheetah (asiatic-cheetah-a). Use `wandb login --relogin` to force relogin


Generating train split:   0%|          | 0/611245 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

Some weights of the model checkpoint at google-bert/bert-base-multilingual-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


  0%|          | 0/19101 [00:00<?, ?it/s]

Epoch 1


  0%|          | 0/19101 [00:00<?, ?it/s]

Epoch 2


In [11]:
model.push_to_hub(
    f"ioai2024japan/{CFG.name}",
    token=userdata.get('hf_write'), private=True
)

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/ioai2024japan/Task2_chizu_007_pretrain/commit/28d2e05ad4114cfa0cb6f00c9910cc28f20d3209', commit_message='Upload BertForMaskedLM', commit_description='', oid='28d2e05ad4114cfa0cb6f00c9910cc28f20d3209', pr_url=None, pr_revision=None, pr_num=None)

{'text': '  𑀫च𑀞च𑀟च𑀟 𑀤च𑀙च 𑀢णच च𑀠𑀲च𑀟𑀢 𑀣च ब𑁦𑀟𑁣पणध𑁦 𑀣𑁣𑀟 𑀞𑁣𑀠च𑀱च 𑀤न𑀱च चलल𑁦ल𑁦𑀳 𑀞𑁣 ढच𑀠ढच𑀟त𑁦षढच𑀠ढच𑀟त𑁦𑀟 𑀣च 𑀠नपन𑀠 𑀞𑁦 ञचन𑀞च च त𑀢𑀞𑀢𑀟 𑀱च𑀟𑀢 पच𑀞च𑀠च𑀢𑀠च𑀟 𑀤नढ𑀢𑀟 𑀫चल𑀢पपच 𑀞𑁣 𑀱न𑀪𑀢𑀟 𑀞𑀱चण𑁣ण𑀢𑀟 𑀫चल𑀢पपच𑀯'}

  0%|          | 0/38202 [00:00<?, ?it/s]

3.5377562046051025
3.9344356060028076
4.249579906463623
3.679966926574707
3.3706040382385254
3.4204628467559814
3.802290678024292
3.488743543624878
3.5119121074676514
3.174436330795288
3.3415582180023193
3.3312113285064697
3.0674831867218018
3.385284185409546
3.360445499420166
2.657778024673462
3.070737361907959
3.2521231174468994
3.2197556495666504
2.872149705886841
3.030869483947754
2.966035842895508
2.992466926574707
2.805521249771118
2.7037947177886963
2.885369300842285
3.144047260284424
3.0346128940582275


KeyboardInterrupt: 

# Data

In [ ]:
# run the trained model on a dev/test split
data_split = "dev"
eval_out = trainer.predict(tokenized_data[data_split])
predictions = eval_out.predictions.argmax(1)
labels = eval_out.label_ids
dev_f1 = f1.compute(predictions=predictions, references=labels, average='macro')

NameError: name 'trainer' is not defined

In [ ]:
# 精度をみる

print(dev_f1)
print()
print(predictions)
print(labels)

total = 0
correct = 0
for i in range(len(predictions)):
  if predictions[i] == labels[i]:
    correct += 1
  total += 1
print()
print(correct/total)

In [ ]:
# write the predictions to a file
with open('{}_predictions.txt'.format(data_split), 'w') as outfile:
  outfile.write('\n'.join([str(p) for p in predictions.tolist()]))

In [ ]:
import gc
import torch

def flush():
  gc.collect()
  torch.cuda.empty_cache()
flush()